In [1]:
#Import des bibliothèque NumPy et Pandas

import numpy as np # https://numpy.org/
import pandas as pd # https://pandas.pydata.org/

In [2]:
#Importation des fichiers nettoyer

datanutri = pd.read_csv("Data/processed_data/sous_nutrition_v2.csv")
datapop = pd.read_csv("Data/processed_data/population_V2.csv")
dataaide = pd.read_csv("Data/processed_data/aide_alimentaire_V2.csv")
datadispo = pd.read_csv("Data/processed_data/dispo_alimentaire_V2.csv")

# 1) Demandes de Marc

### 1- La proportion de personnes en état de sous-nutrition :

#### Jointure

In [3]:
# Jointure des DataFrame "datanutri" et "datapop" dans un seul DataFrame
# pour regrouper les données de la population en sous nutrition  et de la population total

dfproportion = pd.merge(datanutri, datapop, on=("Pays", "Annee"), how='inner')
dfproportion.head()

,Pays,Annee,Population_x,Population_y
0,Afghanistan,2013,8600000,32269589
1,Afghanistan,2014,8800000,33370794
2,Afghanistan,2015,8900000,34413603
3,Afghanistan,2016,9700000,35383032
4,Afghanistan,2017,10500000,36296113


In [4]:
# Changer le titre des colones pour une meilleur compréhention entre la "Population Total" et la "Population en sous nutrition"

dfproportion = dfproportion.rename(columns={'Population_x':'Pop_sous_nutrition','Population_y':'Pop_total'})

In [5]:
# Filtrer les données sur la colones "Année" sur l'année "2017"

pop17 = dfproportion.loc[dfproportion['Annee'] == 2017, :]
pop17.head()

,Pays,Annee,Pop_sous_nutrition,Pop_total
4,Afghanistan,2017,10500000,36296113
10,Afrique du Sud,2017,3100000,57009756
16,Albanie,2017,100000,2884169
22,Algérie,2017,1300000,41389189
28,Allemagne,2017,0,82658409


In [6]:
# Calcul de la proportion de population en sous nutrition

cal = round((pop17['Pop_sous_nutrition']/pop17['Pop_total'])*100,2)

# Insertion de la colone "Proportion de sous nutrition (en %)" avec le resultat du calcul

pop17.insert(4,"Proportion de sous nutrition (en %)",cal,allow_duplicates=False)


In [7]:
# Population total mondial en 2017

total_pop = pop17['Pop_total'].sum()

# Population total mondial en sous nutrition en 2017

total_pop_sous_nutrition = pop17['Pop_sous_nutrition'].sum()

#### Resultat 1.1

In [8]:
pop17.head()

,Pays,Annee,Pop_sous_nutrition,Pop_total,Proportion de sous nutrition (en %)
4,Afghanistan,2017,10500000,36296113,28.93
10,Afrique du Sud,2017,3100000,57009756,5.44
16,Albanie,2017,100000,2884169,3.47
22,Algérie,2017,1300000,41389189,3.14
28,Allemagne,2017,0,82658409,0.00


In [9]:
# Proportion sous nutrition mondial

cal_proportion_mondial = round((total_pop_sous_nutrition/total_pop)*100,2)

print("Proportion de la population mondial en sous nutrition : ", cal_proportion_mondial, "%")

Proportion de la population mondial en sous nutrition :  7.11 %


### 2- Le nombre théorique de personnes qui pourraient être nourries :

In [10]:
# Création d'un DataFrame pour la disponibilité alimentaire

datadispo_alim = datadispo[['Pays', 'Disponibilité alimentaire (Kcal/personne/jour)']]
datadispo_alim = datadispo_alim.groupby('Pays').sum()
datadispo_alim.head()

,Disponibilité alimentaire (Kcal/personne/jour)
Pays,
Afghanistan,2087.0
Afrique du Sud,3020.0
Albanie,3188.0
Algérie,3293.0
Allemagne,3503.0


In [11]:
# Merge entre la disponibilité alimentaire et la population en 2017

pop_dispo = pd.merge(datadispo_alim, pop17, on=("Pays"), how='inner')
pop_dispo = pop_dispo[['Pays', 'Disponibilité alimentaire (Kcal/personne/jour)', 'Pop_total']]
pop_dispo.head()

,Pays,Disponibilité alimentaire (Kcal/personne/jour),Pop_total
0,Afghanistan,2087.0,36296113
1,Afrique du Sud,3020.0,57009756
2,Albanie,3188.0,2884169
3,Algérie,3293.0,41389189
4,Allemagne,3503.0,82658409


#### Resultat 1.2

In [12]:
# Calcul du  nombre de kcal par jour et par personne dispo , multiplier par le nombre de personne

kcalDispo = round(((pop_dispo["Disponibilité alimentaire (Kcal/personne/jour)"] * pop_dispo["Pop_total"])).sum())
# 2500 : nombre de calorie moyen pour un adulte
nb_pers_nourri = round(kcalDispo/2500)

print("Le nombre théorique de personne qui pourrais être nourri est de",nb_pers_nourri,"personnes")

Le nombre théorique de personne qui pourrais être nourri est de 8367593844 personnes


In [13]:
# Calcul de la proportion de personnes pouvant etre théoriquement nourrie grace à la disponibilité alimentaire de 2017

proportion_nourri = round((nb_pers_nourri/total_pop)*100)

print("Grace à la disponibilité de l'année 2017 nous pourrions nourrire",proportion_nourri,"% de la population mondiale en 2017")

Grace à la disponibilité de l'année 2017 nous pourrions nourrire 111 % de la population mondiale en 2017


In [14]:
# Création d'un DataFrame seulement avec les données importante

## pers_nourrie = datadispo[['Pays', 'Produit', 'Origine', 'Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité intérieure (en tonne)']]

# Calcul du nombre théorique de personnes qui pourraient être nourries

## cal_pers_nourrie = round((pers_nourrie['Disponibilité intérieure (en tonne)']/pers_nourrie['Disponibilité alimentaire en quantité (kg/personne/an)'])*1000)

# Insertion de la colones "Nb personne nourrie (théorique)" avec le résultat du calcul

## pers_nourrie.insert(5,"Nb personne nourrie (théorique)",cal_pers_nourrie,allow_duplicates=False)

In [15]:
## pers_nourrie.head()

### 3- Le nombre théorique de personnes qui pourraient être nourries avec disponibilité alimentaire des produits végétaux :

In [16]:
# Création d'un DataFrame pour la disponibilité alimentaire des produits végétaux

datadispo_vege = datadispo[['Pays', 'Origine', 'Disponibilité alimentaire (Kcal/personne/jour)']]
datadispo_veg = datadispo_vege['Origine'] == 'vegetale'
dispo_vegetal = datadispo_vege[datadispo_veg].groupby('Pays').sum()
dispo_vegetal.head()

,Disponibilité alimentaire (Kcal/personne/jour)
Pays,
Afghanistan,1871.0
Afrique du Sud,2533.0
Albanie,2203.0
Algérie,2915.0
Allemagne,2461.0


In [17]:
# Merge entre la disponibilité alimentaire des produits végétaux et la population en 2017

pop_dispo_veg = pd.merge(dispo_vegetal, pop17, on=("Pays"), how='inner')
pop_dispo_veg = pop_dispo_veg[['Pays', 'Disponibilité alimentaire (Kcal/personne/jour)', 'Pop_total']]
pop_dispo_veg = pop_dispo_veg.rename(columns={'Disponibilité alimentaire (Kcal/personne/jour)':'Disponibilité alimentaire vegetale (Kcal/personne/jour)'})
pop_dispo_veg.head()

,Pays,Disponibilité alimentaire vegetale (Kcal/personne/jour),Pop_total
0,Afghanistan,1871.0,36296113
1,Afrique du Sud,2533.0,57009756
2,Albanie,2203.0,2884169
3,Algérie,2915.0,41389189
4,Allemagne,2461.0,82658409


#### Resultat 1.3

In [18]:
# Calcul du  nombre de kcal par jour et par personne dispo , multiplier par le nombre de personne

kcalDispo_veg = round(((pop_dispo_veg["Disponibilité alimentaire vegetale (Kcal/personne/jour)"] * pop_dispo_veg["Pop_total"])).sum())
# 2500 : nombre de calorie moyen pour un adulte
nb_pers_nourri_veg = round(kcalDispo_veg/2500)

print("Le nombre théorique de personne qui pourrais etre nourri, seulement grace à la disponibilité alimentaire des produits végétaux, est de",nb_pers_nourri_veg,"personnes")

Le nombre théorique de personne qui pourrais etre nourri, seulement grace à la disponibilité alimentaire des produits végétaux, est de 6904305679 personnes


In [19]:
# Calcul de la proportion de personnes pouvant etre théoriquement nourrie grace à la disponibilité alimentaire des produits végétaux de 2017

proportion_nourri_veg = round((nb_pers_nourri_veg/total_pop)*100)

print("Grace à la disponibilité alimentaire des produits végétaux de l'année 2017 nous pourrions nourrire",proportion_nourri_veg,"% de la population mondiale en 2017")

Grace à la disponibilité alimentaire des produits végétaux de l'année 2017 nous pourrions nourrire 92 % de la population mondiale en 2017


### 4- Proportions des disponibilités :

In [20]:
# Création d'un DataFrame avec seulement les données importante

proportion_dispo = datadispo[['Pays', 'Produit', 'Origine', 'Aliments pour animaux (en tonne)', 'Pertes (en tonne)', 'Nourriture (en tonne)']]

# Calcul de la disponibilité total

cal_dispo_total = datadispo['Disponibilité intérieure (en tonne)']
##cal_dispo_total = datadispo['Importations - Quantité (en tonne)'] + datadispo['Production (en tonne)']

# Insertion de la colone "Dispo Total (en tonne)" avec le résultat du calcul

proportion_dispo.insert(6,"Dispo Total (en tonne)",cal_dispo_total,allow_duplicates=False)
proportion_dispo.head()

,Pays,Produit,Origine,Aliments pour animaux (en tonne),Pertes (en tonne),Nourriture (en tonne),Dispo Total (en tonne)
0,Afghanistan,Abats Comestible,animale,0.0,0.0,53.0,53.0
1,Afghanistan,"Agrumes, Autres",vegetale,0.0,2.0,39.0,41.0
2,Afghanistan,Aliments pour enfants,vegetale,0.0,0.0,2.0,2.0
3,Afghanistan,Ananas,vegetale,0.0,0.0,0.0,0.0
4,Afghanistan,Bananes,vegetale,0.0,0.0,82.0,82.0


In [21]:
# Regroupement des données par pays et donc somme des colones par pays

proportion_dispo = proportion_dispo.groupby('Pays').sum()

# Passage en DataFrame

pd.DataFrame(proportion_dispo)

,Aliments pour animaux (en tonne),Pertes (en tonne),Nourriture (en tonne),Dispo Total (en tonne)
Pays,,,,
Afghanistan,768.0,1135.0,10735.0,13515.0
Afrique du Sud,5309.0,2193.0,29812.0,61256.0
Albanie,660.0,276.0,3476.0,4758.0
Algérie,4352.0,3753.0,31729.0,42630.0
Allemagne,30209.0,3781.0,79238.0,162275.0
...,...,...,...,...
Émirats arabes unis,1174.0,705.0,6548.0,11850.0
Équateur,1200.0,707.0,8574.0,18614.0
États-Unis d'Amérique,148432.0,7162.0,320889.0,777992.0


In [22]:
# Calcul de la proportion des aliments pour animaux par rapport à la disponibilité

cal_proportion_dispo_animaux = round((proportion_dispo['Aliments pour animaux (en tonne)']/proportion_dispo['Dispo Total (en tonne)'])*100,2)

# Calcul de la proportion des pertes par rapport à la disponibilité

cal_proportion_dispo_pertes = round((proportion_dispo['Pertes (en tonne)']/proportion_dispo['Dispo Total (en tonne)'])*100,2)

# Calcul de la proportion dela nourriture par rapport à la disponibilité

cal_proportion_dispo_nourriture = round((proportion_dispo['Nourriture (en tonne)']/proportion_dispo['Dispo Total (en tonne)'])*100,2)

In [23]:
# Insertion de la colone "Proportion des aliments pour animaux par rapport à la disponibilité (en %)" avec le resultat du calcul

proportion_dispo.insert(4,"Proportion des aliments pour animaux par rapport à la disponibilité (en %)",cal_proportion_dispo_animaux,allow_duplicates=False)

# Insertion de la colone "Proportion des pertes par rapport à la disponibilité (en %)" avec le resultat du calcul

proportion_dispo.insert(5,"Proportion des pertes par rapport à la disponibilité (en %)",cal_proportion_dispo_pertes,allow_duplicates=False)

# Insertion de la colone "Proportion dela nourriture par rapport à la disponibilité (en %)" avec le resultat du calcul

proportion_dispo.insert(6,"Proportion dela nourriture par rapport à la disponibilité (en %)",cal_proportion_dispo_nourriture,allow_duplicates=False)

#### Resultat 1.4

In [24]:
proportion_dispo.head(10)

,Aliments pour animaux (en tonne),Pertes (en tonne),Nourriture (en tonne),Dispo Total (en tonne),Proportion des aliments pour animaux par rapport à la disponibilité (en %),Proportion des pertes par rapport à la disponibilité (en %),Proportion dela nourriture par rapport à la disponibilité (en %)
Pays,,,,,,,
Afghanistan,768.0,1135.0,10735.0,13515.0,5.68,8.40,79.43
Afrique du Sud,5309.0,2193.0,29812.0,61256.0,8.67,3.58,48.67
Albanie,660.0,276.0,3476.0,4758.0,13.87,5.80,73.06
Algérie,4352.0,3753.0,31729.0,42630.0,10.21,8.80,74.43
Allemagne,30209.0,3781.0,79238.0,162275.0,18.62,2.33,48.83
Angola,9259.0,4799.0,15059.0,32029.0,28.91,14.98,47.02
Antigua-et-Barbuda,0.0,0.0,58.0,79.0,0.00,0.00,73.42
Arabie saoudite,11841.0,1040.0,17956.0,31274.0,37.86,3.33,57.42
Argentine,7890.0,3522.0,32531.0,114369.0,6.90,3.08,28.44


In [25]:
# Proportions mondial

prop_mondial_animaux = round((proportion_dispo['Aliments pour animaux (en tonne)'].sum()/proportion_dispo['Dispo Total (en tonne)'].sum())*100,2)
prop_mondial_pertes = round((proportion_dispo['Pertes (en tonne)'].sum()/proportion_dispo['Dispo Total (en tonne)'].sum())*100,2)
prop_mondial_nourriture = round((proportion_dispo['Nourriture (en tonne)'].sum()/proportion_dispo['Dispo Total (en tonne)'].sum())*100,2)

print("Proportion mondial des aliments pour animaux par rapport à la disponibilité : ", prop_mondial_animaux,"%")
print("Proportion mondial des pertes par rapport à la disponibilité : ", prop_mondial_pertes,"%")
print("Proportion mondial de la nourriture par rapport à la disponibilité : ",prop_mondial_nourriture,"%")

Proportion mondial des aliments pour animaux par rapport à la disponibilité :  13.24 %
Proportion mondial des pertes par rapport à la disponibilité :  4.61 %
Proportion mondial de la nourriture par rapport à la disponibilité :  49.51 %


# 2) Demandes de Mélanie

### 1- Les pays pour lesquels la proportion de personnes sous-alimentées est la plus forte en 2017 :

#### Resultat 2.1

In [26]:
# Reprise du DataFrame répondant à la première question de Marc
# avec classement dans l'ordre décroissans sur la colone "Proportion de sous nutrition (en %)"

pop17.sort_values(by=['Proportion de sous nutrition (en %)'], ascending=False).head(10)

,Pays,Annee,Pop_sous_nutrition,Pop_total,Proportion de sous nutrition (en %)
310,Dominique,2017,50000,71458,69.97
472,Haïti,2017,5300000,10982366,48.26
946,République populaire démocratique de Corée,2017,12000000,25429825,47.19
988,Saint-Vincent-et-les Grenadines,2017,50000,109827,45.53
592,Kiribati,2017,50000,114158,43.80
652,Madagascar,2017,10500000,25570512,41.06
622,Libéria,2017,1800000,4702226,38.28
604,Lesotho,2017,800000,2091534,38.25
1102,Tchad,2017,5700000,15016753,37.96
970,Rwanda,2017,4200000,11980961,35.06


### 2- Pays qui ont le plus bénéficié d’aide depuis 2013 :

In [27]:
# Regroupement des données d'aide alimentaire par "Pays" et calcul de la somme de l'aide part pays

aide = dataaide.groupby('Pays')['Quantite (tonne)'].sum()

# Passage en DataFrame

DFaide = pd.DataFrame(aide,columns = ['Quantite (tonne)'])

#### Resultat 2.2

In [28]:
# Affichage dans l'odre décroissant des 10 pays qui ont reçu le plus d'aide

DFaide.sort_values(by=['Quantite (tonne)'], ascending=False).head(10)

,Quantite (tonne)
Pays,
République arabe syrienne,1858943
Éthiopie,1381294
Yémen,1206484
Soudan du Sud,695248
Soudan,669784
Kenya,552836
Bangladesh,348188
Somalie,292678
République démocratique du Congo,288502


### Disponibilité par habitant

In [29]:
# Regroupement par "Pays" et calcul de la somme de la "Disponibilité intérieure (en tonne)" par pays

dispo = datadispo.groupby('Pays')['Disponibilité alimentaire (Kcal/personne/jour)'].sum()

# Passage en DataFrame

DFdispo = pd.DataFrame(dispo, columns = ['Disponibilité alimentaire (Kcal/personne/jour)'])

# Convertion de float vers int

DFdispo['Disponibilité alimentaire (Kcal/personne/jour)'] = DFdispo['Disponibilité alimentaire (Kcal/personne/jour)'].astype(int)

# Affichage

DFdispo.sort_values(by=['Disponibilité alimentaire (Kcal/personne/jour)'], ascending=False).head()

,Disponibilité alimentaire (Kcal/personne/jour)
Pays,
Autriche,3770
Belgique,3737
Turquie,3708
États-Unis d'Amérique,3682
Israël,3610


#### Jointure

In [30]:
# Jointure des DataFrame "DFdispo" et "pop17" dans un seul DataFrame
# pour regrouper les données de la population et de la disponibilité

dispo_habitant = pd.merge(DFdispo, pop17, on=("Pays"), how='inner')

# Selection des colones importante à l'étude

dispo_habitant = dispo_habitant[['Pays', 'Disponibilité alimentaire (Kcal/personne/jour)', 'Pop_total']]

# Affichage de la disponibilité total et population tottal par pays

dispo_habitant

,Pays,Disponibilité alimentaire (Kcal/personne/jour),Pop_total
0,Afghanistan,2087,36296113
1,Afrique du Sud,3020,57009756
2,Albanie,3188,2884169
3,Algérie,3293,41389189
4,Allemagne,3503,82658409
...,...,...,...
167,Émirats arabes unis,3275,9487203
168,Équateur,2346,16785361
169,États-Unis d'Amérique,3682,325084756
170,Éthiopie,2129,106399924


In [31]:
# Calcul de la disponibilité par habitant

## cal_dispo_habitant = (dispo_habitant['Disponibilité alimentaire (Kcal/personne/jour)']/dispo_habitant['Pop_total'])

# Insertion de la colone "Dispo/Habitant (en kg)" avec le calcul

## dispo_habitant.insert(3,"Dispo/Habitant (en kal)",cal_dispo_habitant,allow_duplicates=False)

### 3- Pays ayant le plus de disponibilité/habitant :

#### Resultat 2.3

In [32]:
# Affichage des pays trié dans l'ordre décroissant sur la colone "Dispo/Habitant (en kg)"

dispo_habitant.sort_values(by=['Disponibilité alimentaire (Kcal/personne/jour)'], ascending=False).head()
# Disponibilité alimentaire (Kcal/personne/jour)

,Pays,Disponibilité alimentaire (Kcal/personne/jour),Pop_total
11,Autriche,3770,8819901
16,Belgique,3737,11419748
157,Turquie,3708,81116450
169,États-Unis d'Amérique,3682,325084756
74,Israël,3610,8243848


### 4- Pays ayant le moins de disponibilité/habitant :

#### Resultat 2.4

In [33]:
# Affichage des pays trié dans l'ordre croissant sur la colone "Dispo/Habitant (en kg)"

dispo_habitant.sort_values(by=['Disponibilité alimentaire (Kcal/personne/jour)'], ascending=True).head()
# Disponibilité alimentaire (Kcal/personne/jour)

,Pays,Disponibilité alimentaire (Kcal/personne/jour),Pop_total
127,République centrafricaine,1879,4596023
164,Zambie,1924,16853599
91,Madagascar,2056,25570512
0,Afghanistan,2087,36296113
65,Haïti,2089,10982366


## Utilisation des céréales

In [34]:
# "Blé et produits", "Riz et produits", "Orge et produits", "Maïs et produits", "Seigle et produits", "Avoine", "Millet et produits", "Sorgho et produits", "Céréales, Autres"]

In [35]:
cereales = ['Blé','Riz','Orge','Maïs','Seigle','Avoine','Millet','Sorgho','Céréales, Autres']
dispo_cereale = datadispo.loc[datadispo["Produit"].isin(cereales)].sum()
dispo_cereale = pd.DataFrame(dispo_cereale)
dispo_cereale[10:12]

,0
Nourriture (en tonne),651724.0
Aliments pour animaux (en tonne),839941.0


## Utilisation du manioc par la Thailande

In [36]:
# Création d'un DataFrame pour les disponibilité de la Thailand

dispo_thaï = datadispo[datadispo['Pays'] == 'Thaïlande']
dispo_thaï_produit = dispo_thaï.groupby('Produit').sum()

# Les produits ayant le plus de production en Thailand

dispo_thaï_produit.sort_values(by=['Production (en tonne)'], ascending=False).head()

,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure (en tonne),Exportations - Quantité (en tonne),Importations - Quantité (en tonne),Nourriture (en tonne),Aliments pour animaux (en tonne),Autres Utilisations (en tonne),Pertes (en tonne),Production (en tonne),Semences (en tonne),Traitement (en tonne),Variation de stock
Produit,,,,,,,,,,,,,,,
"Sucre, canne",49.0,61.11,0.33,0.17,100096.0,0.0,0.0,4095.0,0.0,0.0,1001.0,100096.0,0.0,95000.0,0.0
Manioc,40.0,13.00,0.05,0.14,6264.0,25214.0,1250.0,871.0,1800.0,2081.0,1511.0,30228.0,0.0,0.0,0.0
Riz (Eq Blanchi),1139.0,114.57,2.50,19.97,13605.0,6860.0,50.0,7677.0,2405.0,0.0,1786.0,24054.0,606.0,1130.0,-3638.0
Sucre Eq Brut,368.0,37.69,0.00,0.00,2527.0,6462.0,25.0,2526.0,0.0,0.0,0.0,10024.0,0.0,1.0,-1060.0
"Fruits, Autres",63.0,57.56,0.56,0.72,3791.0,2671.0,261.0,3857.0,0.0,0.0,396.0,6141.0,0.0,6.0,60.0


In [37]:
# Les produit étant le plus importer en Thailand

dispo_thaï_produit.sort_values(by=['Importations - Quantité (en tonne)'], ascending=False).head()

,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure (en tonne),Exportations - Quantité (en tonne),Importations - Quantité (en tonne),Nourriture (en tonne),Aliments pour animaux (en tonne),Autres Utilisations (en tonne),Pertes (en tonne),Production (en tonne),Semences (en tonne),Traitement (en tonne),Variation de stock
Produit,,,,,,,,,,,,,,,
Blé,76.0,10.93,0.23,2.18,1882.0,237.0,2118.0,733.0,1130.0,0.0,0.0,2.0,0.0,20.0,0.0
Soja,22.0,2.13,0.98,1.96,1860.0,12.0,1682.0,143.0,0.0,194.0,93.0,190.0,3.0,1428.0,0.0
Poissons Pelagiques,30.0,9.92,1.33,4.15,665.0,1390.0,1460.0,665.0,0.0,0.0,0.0,595.0,0.0,0.0,0.0
Manioc,40.0,13.00,0.05,0.14,6264.0,25214.0,1250.0,871.0,1800.0,2081.0,1511.0,30228.0,0.0,0.0,0.0
Lait - Excl Beurre,45.0,29.35,1.74,2.46,2147.0,200.0,1241.0,1967.0,0.0,147.0,33.0,1095.0,0.0,0.0,11.0


In [38]:
# Création d'un DataFrame pour connaitre les information de la disponibilité de manioc au sein de la Thailand

dispo_manioc_thaï = dispo_thaï[dispo_thaï['Produit'] == 'Manioc']
dispo_manioc_thaï

,Pays,Produit,Origine,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure (en tonne),Exportations - Quantité (en tonne),Importations - Quantité (en tonne),Nourriture (en tonne),Aliments pour animaux (en tonne),Autres Utilisations (en tonne),Pertes (en tonne),Production (en tonne),Semences (en tonne),Traitement (en tonne),Variation de stock
13809,Thaïlande,Manioc,vegetale,40.0,13.0,0.05,0.14,6264.0,25214.0,1250.0,871.0,1800.0,2081.0,1511.0,30228.0,0.0,0.0,0.0


In [39]:
# Création d'un DataFrame pour connaitre les informations concernant la population en Thailand

pop_thaï = pop17[pop17['Pays'] == 'Thaïlande']
pop_thaï

,Pays,Annee,Pop_sous_nutrition,Pop_total,Proportion de sous nutrition (en %)
1114,Thaïlande,2017,6200000,69209810,8.96
